In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
%cd /gdrive/My\ Drive/Fellowship

/gdrive/My Drive/Fellowship


In [0]:
import io
import json
import lmdb
import sqlite3
import pandas as pd
from PIL import Image
from IPython.display import display

## Downloading required files

In [5]:
%%time 
#Might take ~7 minitues
!apt install sqlite3 >/dev/null
!wget https://media.githubusercontent.com/media/kyamagu/paperdoll/master/data/chictopia/chictopia.sql.gz  >/dev/null
!gunzip -c chictopia.sql.gz | sqlite3 chictopia.sqlite3
!wget https://media.githubusercontent.com/media/eBay/modanet/master/annotations/modanet2018_instances_train.json
!cp -r ./drive/My\ Drive/Fellowship/photos.lmdb/ . #wget http://vision.is.tohoku.ac.jp/chictopia2/photos.lmdb.tar



--2019-07-01 16:44:57--  https://media.githubusercontent.com/media/kyamagu/paperdoll/master/data/chictopia/chictopia.sql.gz
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 457165725 (436M) [application/octet-stream]
Saving to: ‘chictopia.sql.gz’

chictopia.sql.gz    100%[===================>] 435.99M   207MB/s    in 2.1s    

2019-07-01 16:45:05 (207 MB/s) - ‘chictopia.sql.gz’ saved [457165725/457165725]

--2019-07-01 16:47:54--  https://media.githubusercontent.com/media/eBay/modanet/master/annotations/modanet2018_instances_train.json
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|151.101.0.133|:443... conne

## Generating Modanet labels from provided paperdoll .sql and provided ModaNet .json

In [6]:
db = sqlite3.connect('file:chictopia.sqlite3?mode=ro', uri=True)

photos = pd.read_sql("""
    SELECT
        *,
        'http://images2.chictopia.com/' || path AS url
    FROM photos
    WHERE photos.post_id IS NOT NULL AND file_file_size IS NOT NULL
""", con=db)
print('photos = %d' % (len(photos)))
photos.head()

photos = 1100987


,id,post_id,path,status,file_file_name,file_file_size,file_content_type,file_updated_at,width,height,created_at,updated_at,url
0,1,61,/photos/thefashbot/1930972049/1930972049_400.jpg,2,open-uri20120907-29328-3yd0mp,122564,image/jpeg,2012-09-07 23:07:38.719079,400,600,2012-09-07 23:07:38.441357,2012-09-07 23:07:39.456904,http://images2.chictopia.com//photos/thefashbo...
1,2,73,/photos/thefashbot/3613535083/3613535083_400.jpg,2,open-uri20120907-29328-1szdum6,119821,image/jpeg,2012-09-07 23:07:45.966209,400,600,2012-09-07 23:07:45.721208,2012-09-07 23:07:46.438801,http://images2.chictopia.com//photos/thefashbo...
2,3,73,/photos/thefashbot/3613535083/3420751783/34207...,2,open-uri20120907-29328-1uz77bi,115073,image/jpeg,2012-09-07 23:07:46.825511,400,600,2012-09-07 23:07:45.785651,2012-09-07 23:07:47.722063,http://images2.chictopia.com//photos/thefashbo...
3,4,85,/photos/thefashbot/10902372197/10902372197_400...,2,open-uri20120907-29328-1nqep2j,115259,image/jpeg,2012-09-07 23:07:53.314908,400,600,2012-09-07 23:07:52.920956,2012-09-07 23:07:54.329817,http://images2.chictopia.com//photos/thefashbo...
4,5,98,/photos/thefashbot/4328323416/4328323416_400.jpg,2,open-uri20120907-29328-10ax6ou,123235,image/jpeg,2012-09-07 23:08:00.645646,400,600,2012-09-07 23:08:00.081276,2012-09-07 23:08:01.889023,http://images2.chictopia.com//photos/thefashbo...


In [0]:
with open('modanet2018_instances_train.json') as json_file:  
    data = json.load(json_file)

In [0]:
data.keys()

dict_keys(['info', 'images', 'year', 'licenses', 'type', 'annotations', 'categories'])

In [0]:
data['categories']

[{'id': 1, 'name': 'bag', 'supercategory': 'fashion'},
 {'id': 2, 'name': 'belt', 'supercategory': 'fashion'},
 {'id': 3, 'name': 'boots', 'supercategory': 'fashion'},
 {'id': 4, 'name': 'footwear', 'supercategory': 'fashion'},
 {'id': 5, 'name': 'outer', 'supercategory': 'fashion'},
 {'id': 6, 'name': 'dress', 'supercategory': 'fashion'},
 {'id': 7, 'name': 'sunglasses', 'supercategory': 'fashion'},
 {'id': 8, 'name': 'pants', 'supercategory': 'fashion'},
 {'id': 9, 'name': 'top', 'supercategory': 'fashion'},
 {'id': 10, 'name': 'shorts', 'supercategory': 'fashion'},
 {'id': 11, 'name': 'skirt', 'supercategory': 'fashion'},
 {'id': 12, 'name': 'headwear', 'supercategory': 'fashion'},
 {'id': 13, 'name': 'scarf/tie', 'supercategory': 'fashion'}]

In [0]:
categories = {1:  'bag',2: 'belt', 3: 'boots', 4:  'footwear', 5: 'outer', 6: 'dress', 7: 'sunglasses', 
              8:  'pants',  9: 'top', 10: 'shorts', 11: 'skirt', 12: 'headwear', 13: 'scarf/tie'}

In [0]:
labels = pd.DataFrame.from_dict(data['annotations'])
labels.head()


,area,bbox,category_id,id,image_id,iscrowd,segmentation
0,1746,"[160, 247, 97, 18]",2,0,736791,0,"[[161, 248, 170, 248, 173, 249, 189, 251, 199,..."
1,1260,"[287, 537, 30, 42]",4,1,736791,0,"[[198, 543, 197, 551, 198, 556, 199, 565, 200,..."
2,1092,"[287, 537, 26, 42]",4,2,736791,0,"[[289, 554, 287, 564, 287, 573, 287, 577, 292,..."
3,37352,"[132, 255, 161, 232]",11,3,736791,0,"[[147, 281, 141, 288, 136, 297, 132, 304, 132,..."
4,14605,"[145, 126, 115, 127]",9,4,736791,0,"[[159, 131, 162, 150, 162, 167, 159, 176, 150,..."


In [0]:
modanet_ids = [d['id'] for d in data['images']]
modanet_id_from_paperdoll = list(set(photos.id)&set(modanet_ids))
modanet_labels = photos[photos['id'].isin(list(modanet_id_from_paperdoll))].reset_index(drop=True)

In [0]:
labels.replace({'category_id': categories},inplace = True)

In [0]:
labels.set_index('image_id',inplace = True, drop = True)

In [0]:
labels['category_id'].head()

image_id
736791        belt
736791    footwear
736791    footwear
736791       skirt
736791         top
Name: category_id, dtype: object

In [0]:
grouped_df = labels.groupby('image_id')
# df.groupby('id')['words'].apply(','.join)

In [0]:
';'.join(str(e) for e in list(labels.category_id))

'belt;footwear;footwear;skirt;top;sunglasses;top;skirt;footwear;footwear;top;pants;footwear;footwear;top;scarf/tie;outer;footwear;footwear;bag;footwear;footwear;dress;outer;footwear;footwear;scarf/tie;bag;dress;boots;boots;footwear;footwear;pants;bag;headwear;top;outer;footwear;footwear;bag;skirt;top;footwear;shorts;bag;top;footwear;footwear;bag;dress;belt;dress;footwear;footwear;outer;pants;footwear;footwear;top;outer;pants;footwear;footwear;top;skirt;footwear;footwear;belt;top;top;outer;bag;pants;footwear;footwear;footwear;footwear;shorts;top;sunglasses;top;skirt;footwear;footwear;headwear;top;outer;bag;pants;footwear;footwear;sunglasses;bag;skirt;top;footwear;footwear;outer;bag;dress;footwear;footwear;belt;sunglasses;scarf/tie;dress;dress;boots;boots;bag;outer;pants;bag;outer;dress;footwear;footwear;bag;footwear;footwear;pants;top;belt;top;outer;footwear;footwear;skirt;scarf/tie;headwear;belt;bag;outer;pants;boots;boots;skirt;bag;top;shorts;bag;top;scarf/tie;footwear;footwear;belt;b

In [0]:
columns = ['image_id', 'label']
final_df = pd.DataFrame(columns=columns)

In [0]:
for key, item in tqdm(grouped_df):
    each_image_id = grouped_df.get_group(key)
    label = ';'.join(str(e) for e in list(set(each_image_id.category_id)))
    image_id = set(list(each_image_id.index.get_level_values(level=0))).pop()
#     print(image_id, label)
    final_df  = final_df.append({'image_id':image_id, 'label':label}, ignore_index=True)

100%|██████████| 52254/52254 [04:34<00:00, 190.53it/s]


In [0]:
final_df.head()

,image_id,label
0,3,outer;top;pants;footwear
1,13,outer;top;shorts;headwear;footwear;pants
2,148,top;pants;skirt;footwear
3,156,footwear;dress;belt
4,174,dress;sunglasses;bag;footwear;belt


In [0]:
final_labels = pd.merge(modanet_labels, final_df, on = ['image_id'])

In [25]:
final_labels.head()

,post_id,path,status,file_file_name,file_file_size,file_content_type,file_updated_at,width,height,created_at,updated_at,url,image_id,file_name,label
0,73,/photos/thefashbot/3613535083/3420751783/34207...,2,open-uri20120907-29328-1uz77bi,115073,image/jpeg,2012-09-07 23:07:46.825511,400,600,2012-09-07 23:07:45.785651,2012-09-07 23:07:47.722063,http://images2.chictopia.com//photos/thefashbo...,3,34207517833420751783_400.jpg,pants;footwear;top;outer
1,142,/photos/thefashbot/2353856772/2353856772_400.jpg,2,open-uri20120907-29328-s7gmu,106894,image/jpeg,2012-09-07 23:08:31.073256,400,600,2012-09-07 23:08:30.687548,2012-09-07 23:08:32.031942,http://images2.chictopia.com//photos/thefashbo...,13,23538567722353856772_400.jpg,top;headwear;outer;pants;footwear;shorts
2,79,/photos/poppylee/3083327112/3083327112_400.jpg,2,open-uri20120907-29328-1l7sf9u,124987,image/jpeg,2012-09-07 23:14:50.212592,400,600,2012-09-07 23:14:49.805666,2012-09-07 23:14:50.742101,http://images2.chictopia.com//photos/poppylee/...,148,30833271123083327112_400.jpg,skirt;top;pants;footwear
3,104,/photos/poppylee/8150270690/8150270690_400.jpg,2,open-uri20120907-29328-v7lh6v,126139,image/jpeg,2012-09-07 23:15:06.192652,400,600,2012-09-07 23:15:05.860767,2012-09-07 23:15:06.622103,http://images2.chictopia.com//photos/poppylee/...,156,81502706908150270690_400.jpg,dress;footwear;belt
4,353,/photos/Carolina/7809452194/7809452194_400.jpg,2,open-uri20120907-29328-1x0oi3x,208504,image/jpeg,2012-09-07 23:15:40.865098,400,600,2012-09-07 23:15:39.440233,2012-09-07 23:15:41.682777,http://images2.chictopia.com//photos/Carolina/...,174,78094521947809452194_400.jpg,bag;dress;footwear;sunglasses;belt


## Extracting Paperdoll Dataset

In [0]:
class PhotoData(object):
    def __init__(self, path):
        self.env = lmdb.open(
            path, map_size=2**36, readonly=True, lock=False
        )
        
    def __iter__(self):
        with self.env.begin() as t:
            with t.cursor() as c:
                for key, value in c:
                    yield key, value
        
    def __getitem__(self, index):
        key = str(index).encode('ascii')
        with self.env.begin() as t:
            data = t.get(key)
        if not data:
            return None
        with io.BytesIO(data) as f:
            image = Image.open(f)
            image.load()
            return image
        
    def __len__(self):
        return self.env.stat()['entries']

In [0]:
photo_data = PhotoData('./photos.lmdb')
print(len(photo_data))

1097474


### Extracting the images from Paperdoll dataset and storing the images required by the ModaNet labels

In [0]:
from tqdm import tqdm
for i in tqdm(range(len(modanet_labels)), position = 0):
    try:
        photo = modanet_labels.iloc[i]
        photo_data[photo.id].save(str('modanet_data/'+photo.file_name))
    except:
        image_does_not_exist.append(photo.id)
        continue

  0%|          | 0/52377 [00:00<?, ?it/s]

In [0]:
data.